<a href="https://colab.research.google.com/github/Wisley1/Text_comprassion/blob/code_of_programm/my_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
#@title Imports
!pip install nltk
import textwrap
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
#@title Functions/Clean your sentences
def preprocess_text(text):
    sentences = text.split(". ")
    processed_sentences = []

    for sentence in sentences:
        processed_sentence = sentence.replace("[^a-zA-Z]", " ").split(" ")
        processed_sentences.append(processed_sentence)

    return processed_sentences

In [104]:
#@title Sentence Similarity
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)


In [105]:
#@title Looking for similarity between sentences
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


In [106]:
#@title Generate Summary method
def generate_summary(text, top_n):

    stop_words = stopwords.words('russian')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences = preprocess_text(text)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    #print("Самые значимые слова", *ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize text

    print("Ваш текст:\n", *summarize_text)

In [107]:
print('Введите ваш текст:')
text = input()
print('Сколько предложений должен иметь итоговый текст?')
summary = generate_summary(text, int(input()))
print(summary)

Кроссовки Saucony Shadow 5000. Модель, выпущенная в 1989 году для бегунов, в современном переиздании сочетает в себе комфорт беговых моделей и лучшие традиции спортивного стиля.
Сколько предложений должен иметь итоговый текст?
2
Ваш текст:
 Модель, выпущенная в 1989 году для бегунов, в современном переиздании сочетает в себе комфорт беговых моделей и лучшие традиции спортивного стиля. Кроссовки Saucony Shadow 5000
None
